In [1]:
# Model that predicts which passengers survived the Titanic shipwreck #

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test_ids = test["PassengerId"]

In [4]:
data.head(6)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q


In [5]:
# Cleaning data #
# Removing all string data for later algorithm #
def clean(data):
    data = data.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
    
# Filling missing data in a column with the median value in that column #
    cols = ["Age", "SibSp", "Parch", "Fare"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
        
# Filling empty values in Embarked column with U(nknown) value #
    data.Embarked.fillna("U", inplace=True)
    return data

data = clean(data)
test = clean(test)

In [6]:
data.head(6)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,28.0,0,0,8.4583,Q


In [7]:
# Converting strings to values using preprocessing module from the Scikit-Learn library #
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]
for col in cols:
    data[col] = le.fit_transform(data[col])
    test[col] = le.transform(test[col])
    print(le.classes_)

['female' 'male']
['C' 'Q' 'S' 'U']


In [8]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [9]:
# Using binary classification to predict the outcome #
# Importing a logistic Regression model from the Scikit-Learn library #
# Importing the train_test_split function to split the data into a training set and a validation set #
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Creating and dropping the variable that the model will try to predict #
Y = data["Survived"]
X = data.drop("Survived", axis=1)

# Dividing data into a training set and a validation set #
X_trian, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [10]:
# Creating classifier #
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_trian, Y_train) 

In [11]:
# Using the accuracy_score function to calculate the classification accuracy on the validation set #
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(Y_val, predictions)

0.8100558659217877

In [12]:
# Saving the results #
submission_preds = clf.predict(test)

In [13]:
df = pd.DataFrame({"PassengerId": test_ids.values, 
                  "Survived": submission_preds,
                  })

In [14]:
df.to_csv("submission.csv", index=False)